<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# Hansard parser from XML files saved to disk

In [2]:
def read_xml_from_path(path):
    # Read from path - decode bytes to strings
    with open(path, 'rb') as file:
        xml = file.read().decode(errors='replace') # Replace unrecognised utf-8 tokens with '?'
        
    return xml

In [3]:
def parse_hansard_xml(xml, memberid2personid_lookup, filter_no_speaker=True):
    # Use BeautifulSoup to tidy the XML
    import pandas as pd
    from bs4 import BeautifulSoup
    bs = BeautifulSoup(xml)
    
    # Define a function to parse a chunk of XML for an individual speech
    def parse_speech_xml(speech_xml):
        name = speech_xml.get('speakername')
        speech_id = speech_xml.get('id')
        person_id = speech_xml.get('person_id')
        if pd.isna(person_id):
            member_id = speech_xml.get('speakerid')
            if member_id in memberid2personid_lookup.keys():
                person_id = memberid2personid_lookup[member_id]
        text = speech_xml.find('p')
        if text is not None:
            text = text.get_text()
        return name,speech_id,person_id,text
    
    # Get an XML object for each speech in the debate
    speeches = bs.find_all('speech') 
    
    # Create dictionary to store debate features
    debate_dict = {'name':[],
                   'speech_id':[],
                   'person_id':[],
                   'text':[]}
    
    for speech_xml in speeches:
        name,speech_id,person_id,text = parse_speech_xml(speech_xml)
        if filter_no_speaker: # If we're filtering 'no speaker' lines, then drop speeches with no name
            if name is not None:
                for variable in debate_dict.keys():
                    debate_dict[variable].append(eval(variable))
        else:
            for variable in debate_dict.keys():
                debate_dict[variable].append(eval(variable))
    
    return pd.DataFrame(debate_dict)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
people_df = pd.read_csv('people.csv').drop('Unnamed: 0',axis=1)

lookup = lambda row: {member_id:row.person_id for member_id in eval(row.memberships)}
lookup_list = people_df.progress_apply(lookup, axis=1).to_list()

memberid2personid_lookup = {member_id:person_id for lookup_dict in lookup_list for member_id,person_id in lookup_dict.items()}

target_urls = pd.read_csv('debate_urls.csv').url.to_list()

df = parse_hansard_xml('', memberid2personid_lookup)

for url in tqdm(target_urls):
    filename = 'debates_xml/'+url.split('/')[-1]
    
    speech_xml = read_xml_from_path(filename)
    debate_df = parse_hansard_xml(speech_xml, memberid2personid_lookup)
    
    df = df.append(debate_df)
    
display(df.head())

  0%|                                                                                        | 0/13976 [00:00<?, ?it/s]

 30%|██████████████████████▏                                                   | 4184/13976 [00:00<00:00, 41830.37it/s]

 60%|████████████████████████████████████████████▎                             | 8368/13976 [00:00<00:00, 39700.81it/s]

 88%|████████████████████████████████████████████████████████████████▍        | 12346/13976 [00:00<00:00, 39302.83it/s]

100%|█████████████████████████████████████████████████████████████████████████| 13976/13976 [00:00<00:00, 39748.03it/s]

  0%|                                                                                        | 0/19646 [00:00<?, ?it/s]

  0%|                                                                                        | 0/19646 [00:00<?, ?it/s]

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
date_from_speech_id = lambda x: pd.to_datetime(x.split('/')[-1][:10])

df['speech_date'] = df.speech_id.progress_apply(lambda x: date_from_speech_id(x))

In [ ]:
display(df)

In [ ]:
df.to_csv('hansard_in_full.csv')